In [1]:
import torch

labels = torch.tensor([1, 2, -1, -1])
sim = torch.rand(4, 4)
global_R = torch.full((labels.size(0), labels.size(0)), -1.0)
mask_label = (labels != -1)
label_mask = mask_label.unsqueeze(0) & mask_label.unsqueeze(1)
label_R = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
global_R[label_mask] = label_R[label_mask]

print(mask_label.unsqueeze(0), '\n', mask_label.unsqueeze(1))
print(global_R)

tensor([[ True,  True, False, False]]) 
 tensor([[ True],
        [ True],
        [False],
        [False]])
tensor([[ 1.,  0., -1., -1.],
        [ 0.,  1., -1., -1.],
        [-1., -1., -1., -1.],
        [-1., -1., -1., -1.]])


从上面的输出结果可以看到，`.unsqueeze()`是在指定参数上增加维度

In [2]:
other_mask = ~label_mask
l, u = 0.5, 0.9
global_R[other_mask & (sim >= u)] = torch.tensor(1.0)
global_R[other_mask & (sim <= l)] = torch.tensor(0.0)
print(sim)
print(global_R)

tensor([[0.4763, 0.6616, 0.2477, 0.4547],
        [0.0404, 0.4367, 0.2876, 0.0377],
        [0.7667, 0.4985, 0.9509, 0.6440],
        [0.2946, 0.5130, 0.3965, 0.5472]])
tensor([[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [-1.,  0.,  1., -1.],
        [ 0., -1.,  0., -1.]])


In [4]:
uncert_mask = (global_R == -1)
mask = torch.tril(uncert_mask, diagonal=-1)
row, col = torch.where(mask)
uncert_ij = torch.stack([row, col], dim=1).tolist()
print(uncert_ij)

[[2, 0], [3, 1]]


In [5]:
# 计算损失
pos_mask = (global_R == 1)
neg_mask = (global_R == 0)
eps = 1e-10
pos_entropy = -torch.log(torch.clamp(sim, eps, 1.0)) * pos_mask
neg_entropy = -torch.log(torch.clamp(1 - sim, eps, 1.0)) * neg_mask
loss = pos_entropy.mean() + neg_entropy.mean() + u - l
print(loss)

tensor(0.7474)


## 第1次train梳理

- 矩阵R没有办法拼接，选不确定性样本对必须全局来弄。
- 也就是说，得先得到全局的feats，然后全局计算相似度，主要的开销是这里。

In [ ]:
from tqdm import trange, tqdm
def train(self, args):

    for epoch in trange(args.train_epochs, desc="Training"):

        model.train()
        
        for batch in tqdm(train_semi_dataloader):

            with torch.set_grad_enabled(True):
                # input_ids, attention_mask, label
                feats_batch = model()

                sim_batch = self.get_sim_score(feats_batch)
                R_batch = self.get_R(sim_batch, u, l)

                # 计算相似度损失
                sim_loss = pos_entropy + neg_entropy

                indices_pairs_batch = self.get_uncert_pairs(R_batch)
                text_pairs_batch = self.get_text_pairs(indices_pairs_batch)

                global_R = torch.cat((global_R, R_batch))
                indices_pairs = torch.cat((indices_pairs, indices_pairs_batch))
                text_pairs = torch.cat((text_pairs, text_pairs_batch))

        if epoch % 5 == 0:
            # 调用LLM: dict{pair_index, llm_pred, conf}
            llm_generated_outputs = self.llm_labeling(text_pairs)
            # 更新R
            new_R = self.update_R(llm_generated_outputs, global_R)
            # 更新数据集，第一轮应该是构建数据集
            # triplet_dataset: (anchor, pos, neg)
            # 如果全部这样选负例，会不会导致一开始训练太难了？
            # 因为FaceNet提到了这个问题，LANID则直接随机选取负例
            triplet_dataset = self.update_data(llm_generated_outputs)
            triplet_dataloader = DataLoader(triplet_dataset)

            for batch in triplet_dataloader:
                seq_emb = model(batch)

                tri_loss = self.tri_loss(anchor, pos, neg)

            tri_loss += tri_loss

        loss = sim_loss + tri_loss
        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_step()

## 第2次train梳理

In [ ]:
def train(self, args, ):

    for epoch in trange(args.num_train_epochs, desc="Training"):
        # 1. 相似度损失
        for batch in tqdm(train_semi_dataloader):

            with torch.set_grad_enabled(True):
                # input_ids, attention_mask, label
                feats_batch = model()

                sim_batch = self.get_sim_score(feats_batch)
                R_batch = self.get_R(sim_batch, u, l)

                # 计算相似度损失
                sim_loss = pos_entropy + neg_entropy
            sim_loss += sim_loss
        
        # 2. 三元组损失（每隔几轮更新）
        if (epoch + 1) % 5 == 0:
            # 调用LLM标注，得到标注结果
            # 这里llm_labeling方法中已经包含前期步骤：先拿到**全局**特征/sim/R
            # feats, y_true = self.eval(train_semi_dataloader)
            # sim_mat = self.get_sim_score(feats)
            # global_R = self.get_global_R(sim_mat, u, l)
            # indices_pairs = self.get_uncert_pairs(global_R)
            # text_pairs = self.get_text_pairs(indices_pairs)
            llm_outputs = self.llm_labeling(text_pairs)

            # 更新R矩阵和三元组数据集
            global_R = self.update_R(global_R)
            tri_dataset = self.update_data(llm_outputs)
            
        for batch in tri_dataloader:
            anchor, neg, pos = model()

            tri_loss = self.tri_loss(anchor, pos, neg)

        loss = sim_loss + tri_loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    
        


            